# PHÂN TÍCH CẢM XÚC VỚI LSTMs

- Điểm: 
- Nhận xét:

## Dẫn nhập

### Giới thiệu bài toán
Trong thực trạng nhiều công ty thương mại điện tử phát triển nở rộ ngày nay, việc thu thập đánh giá từ các bình luận của người dùng là rất cần thiết. Các bình luận này có chứa nhiều thông tin quan trọng, có thể giúp các trang web dễ dàng lọc, đề xuất các mặt hàng, địa điểm phù hợp với từng người dùng và đánh giá chất lượng sản phẩm cũng như dịch vụ của các đối tác.

Với số lượng bình luận từ hàng nghìn tới hàng trăm nghìn, việc phân loại các bình luận từ người dùng không phải là điều dễ dàng, và đòi hỏi rất nhiều nhân lực. Cụ thể ở đây, ta quan tâm đến cảm xúc (tích cực/tiêu cực) gắn liền với các bình luận, từ đó suy ra cảm nhận của người dùng và đánh giá chất lượng các sản phẩm.

Cùng sự phát triển của AI và các phương pháp machine learning, hiện nay chúng ta có thể giải quyết bài toán này bằng các mô hình Deep Learning với độ chính xác có thể sánh ngang với khả năng đọc hiểu của con người.

Trong bài tập này, ta sẽ tập trung vào phân tích các bình luận tiếng Việt để tìm ra cảm xúc được thể hiện trong bình luận.
![](https://drive.google.com/uc?export=view&id=16m7rYwl40I0CBJKWlu44CyAXEMflcF9R)
![](https://drive.google.com/uc?export=view&id=1MSEwafiJ7hIoRD4QEzL5umCK8FqlAojN)


### Hướng giải quyết bài toán

Ta sẽ dùng mạng **LSTM** (Long Short-Term Memory) để giải quyết bài toán **Sentiment Analysis**  (phân tích cảm xúc) trên dữ liệu văn bản.

Đầu vào nhận được là một văn bản.

Đầu ra cần trả về là loại cảm xúc được thể hiện trong văn bản: tích cực, tiêu cực, hay trung lập (positive - negative - neutral). Trong phạm vi của bài tập này, chúng ta chỉ quan tâm đến hai cảm xúc positive và negative.

![caption](https://drive.google.com/uc?export=view&id=1cg-W4IXMKP9jfpCcADKXqXwpgEV19I-f)

### Hướng dẫn làm bài
Ở bài tập này, các bạn sẽ được yêu cầu hoàn thành phần code cần thực hiện ở các mục TODO. Ở mỗi mục TODO sẽ có các code cell. Trong đó, các bạn sẽ code phần đáp án của mình giữa 2 phần:
```python
### START CODE HERE ###
<phần bài làm>
### END CODE HERE ###
```

### Cách đánh giá mô hình
Mô hình luyện được sẽ được đánh giá trên tập dữ liệu gồm 3000 bình luận (được giấu nhãn) với độ đo F1 score. 

Các bạn nộp dự đoán của mình qua [Kaggle](https://www.kaggle.com/t/4af2ae58e2294153931a839a75eece16)

### Cách tính điểm

Điểm tổng của bài tập sẽ là:
$$
Score = solution\times{70\%} + test\_score \times{30\%}
$$
Trong đó:
- solution: điểm của phần bài tập được hoàn thành dưới dạng các TODO (thang điểm 100)
- test_score: điểm trên leaderboard của Kaggle, sẽ được so sánh với  baseline model (mô hình mức cơ sở) (thang điểm 100)

## Tìm hướng giải quyết bài toán

### Tập dữ liệu

Trong assignment này, chúng ta sử dụng tập dữ liệu review trên trang web Foody với khoảng 30,000 mẫu được gán nhãn, trong đó có khoảng 15,000 mẫu positive và 15,000 mẫu negative (Nguồn: https://streetcodevn.com/blog/dataset).

Tập dữ liệu này có thể được tải bằng đoạn code bên dưới:

In [ ]:
# Tải data
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1eoMIeJoIdITRsW94IwyAx4O2sDZprBSW', dest_path='./assignment4-data.zip', unzip=True)

Cấu trúc của folder vừa tải xuống:

In [ ]:
data_dir = 'Assignment4'
!ls {data_dir} #Các lệnh bắt đầu bằng ! trong colab code cell tương đương các lệnh trong terminal

Trong đó:
- **train.csv**: file csv chứa 27,000 mẫu dữ liệu đã gán nhãn
- **test.csv**: file chứa 3000 mẫu test giấu nhãn, dùng để dự đoán và nộp kết quả lên kaggle
- **sample_submission.csv**: file mẫu thể hiện hình thức nộp kết quả
- **word_list.npy**: chứa danh sách các từ tiếng Việt đã xử lý
- **word_vectors.npy**: chứa vectors  biểu diễn các từ trong **word_list.npy**

### Phân tích bài toán dưới góc nhìn Word Vectors


Nếu như chúng ta giữ nguyên định dạng kí tự của văn bản đầu vào thì rất khó để thực hiện các thao tác toán học, như tính tích vô hướng (dot product) , đưa vào hàm softmax, hay các thuật toán trên Neural Net như backpropagation.

Vậy nên, thay vì sử dụng chuỗi kí tự, ta sẽ thay đổi  định dạng của văn bản đầu vào bằng các vector biểu diễn, để thuận tiện cho việc tính toán.

![Word2Vec](https://drive.google.com/uc?export=view&id=12xKP6pFzDzj-0ToZX-27RjOhRcMFrfeb)

Trong hình minh hoạ ở trên, dữ liệu đầu vào là câu "Tôi ăn ... rồi" với tổng cộng 16 từ. Giả sử ta có thể mã hóa mỗi tiếng thành một vector D chiều, ví dụ như:

```
Tôi = [0.2, 0, 0.8, 0.5,]
ăn = [0, 0.6, 0.3, -0.1]
.
.
.
rồi = [0.5, 0.2, 0.1, -0.7]
với D = 4
```

Vậy, cả câu "Tôi ăn ... rồi" có thể được biểu diễn bằng một ma trận 16*D chiều.
```
[[0.2, 0, 0.8, 0.5]
[0, 0.6, 0.3, -0.1]
.
.
.
[0.5, 0.2, 0.1, -0.7]]
```

Để ánh xạ từ một **từ** sang một **vector**, chúng ta sử dụng một bộ vector biểu diễn từ đã được chuẩn bị sẵn.

### Import các thư viện cần thiết

In [ ]:
import os
import sys
import time
import datetime
import math

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

In [ ]:
tf.__version__

## Chuẩn bị dữ liệu huấn luyện

### Tải dữ liệu 
Sau đây ta tiến hành tải dữ liệu từ file `train.csv`

In [ ]:
pd.options.display.max_colwidth=1000
train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
train_df.head()

In [ ]:
print('Number of train samples in total:', len(train_df))

In [ ]:
print('Number of positives:', np.sum(train_df['class']==1))
print('Number of negatives:', np.sum(train_df['class']==0))

Dữ liệu gồm 3 cột: `id`, `text` và `class`. Trong đó:
- `id`: id của mẫu dữ liệu
- `text`: dữ liệu văn bản, đã được tiền xử lý để gộp từ và dấu câu
- `class`: nhãn của dữ liệu, 1 là positive, 0 là negative

Để giúp hiểu rõ hơn về dữ liệu, ta có thể hiển thị một số review bất kỳ như sau. Các bạn có thể thử chạy các code cell ở dưới nhiều lần để thấy sự đa dạng của các bình luận trong tập dữ liệu (về độ dài, cách gõ, hành văn, ngôn từ, *vân vân*)

5 review tích cực

In [ ]:
sample_positive = train_df[train_df['class'] == 1].sample(5)
sample_positive

5 review tiêu cực


In [ ]:
sample_negative = train_df[train_df['class'] == 0].sample(5)
sample_negative

Tiếp theo ta load dữ liệu từ file `test.csv`. Chú ý rằng các review trong tập test được giấu nhãn.

In [ ]:
test_df = pd.read_csv(os.path.join(data_dir, 'test.csv'))
test_df.head()

In [ ]:
print('Number of test samples in total:', len(test_df))

### Tải bộ word embeddings (vector biểu diễn từ)
Word embeddings dịch thô là "từ nhúng", hiểu nôm na là nhúng bộ từ vựng vào không gian vector.

Đầu tiên, để có thể biểu diễn một từ bằng một vector, ta sẽ sử dụng mô hình đã được luyện sẵn (pretrained model). Ở đây ta sẽ dùng mô hình tiếng Việt trong fasttext, bộ biểu diễn từ cho 157 ngôn ngữ do Facebook AI Research Lab thực hiện (nguồn: https://fasttext.cc).

Tuy nhiên, kích thước của mô hình luyện sẵn khá lớn (khoảng 1,2 GB). Mô hình luyện sẵn bao gồm khoảng 2 triệu từ, mỗi từ lại được biểu diễn dưới dạng một vector 300 chiều.



> *Câu hỏi phụ: ước tính độ lớn của một ma trận 2D có kích thước 2000000 x 300, mỗi phần tử là một giá trị float. So sánh với kích thước file được cho biết (1,2 GB).*

> **_Các câu hỏi phụ (ở đây và ở dưới) chỉ giúp các bạn tự suy nghĩ và hiểu thêm về bài toán, không cần ghi câu trả lời. Câu hỏi phụ cũng không ảnh hưởng tới điểm bài tập. Điểm bài tập chỉ phụ thuộc vào TODO._**

Kích thước lớn của mô hình gây khó khăn cho việc tải và sử dụng nó cho bài toán của chúng ta. Vì vậy, đội giảng dạy đã thu gọn mô hình trên, với số lượng từ vừa đủ để có thể xử lý tập dữ liệu review đồ ăn của Foody.

In [ ]:
words_list = np.load(os.path.join(data_dir, 'words_list.npy'))
print('Prunned vocabulary loaded!')
words_list = words_list.tolist()
word_vectors = np.load(os.path.join(data_dir, 'word_vectors.npy'))
word_vectors = np.float32(word_vectors)
print ('Word embedding matrix loaded!')

Để đảm bảo dữ liệu được tải một cách chính xác, ta nên kiểm tra xem số từ vựng và số chiều của ma trận biểu diễn từ (word embedding) có khớp với nhau hay không? Trong trường hợp này số từ được giữ lại là 19899 và số chiều của mỗi vector biểu diễn là 300 chiều.

In [ ]:
print('Size of the vocabulary: ', len(words_list))
print('Size of the word embedding matrix: ', word_vectors.shape)

### Cách lấy word vector của một từ

`words_list` và `word_vectors` đã được thiết lập sẵn sao cho vector đầu tiên trong `word_vectors` sẽ tương ứng với từ đầu tiên `words_list`. Tương tự với cặp vector và từ thứ hai, thứ ba, vân vân, đến cuối.


In [ ]:
for i in range(5):
  print('Từ',  words_list[i], 'có chỉ số', i)
  print('Biểu diễn vector của từ', words_list[i], 'có 10 giá trị đầu tiên là', word_vectors[i][:10])
  print()

Theo như trên, để truy xuất vector một từ nhất định thì ta cần biết chỉ số của từ đó. Lấy ví dụ trường hợp ta muốn biết biểu diễn vector của từ "ngon" thì ta cần thực hiện các bước sau:


In [ ]:
# Bước 1
word = 'ngon'
  
# Bước 2
word_idx = words_list.index(word)

# Bước 3
word_vec = word_vectors[word_idx]

Tuy nhiên, phương pháp trên khá chậm, và sẽ trở nên rất chậm khi ta cần truy xuất hàng trăm, hàng nghìn vector của các từ trong câu. Khúc mắc nằm ở **bước 2**: việc tìm index của một từ trong một list (không có phần tử trùng) có độ phức tạp O(n). Sử dụng dictionary trong python sẽ giảm độ phức tạp xuống còn O(1),  giúp tăng tốc quá trình này

In [ ]:
word2idx = {w:i for i,w in enumerate(words_list)}
print(word2idx)

In [ ]:
# Bước 1
word = 'ngon'

# Bước 2
word_idx = word2idx[word]

# Bước 3
word_vec = word_vectors[word_idx]

Thông qua dictionary `word2idx`, việc truy xuất chỉ số của một từ bất kì đã giảm độ phức tạp xuống còn O(1). Việc này sẽ giúp tăng tốc quá trình biến đổi tập dữ liệu của chúng ta ở dưới.


In [ ]:
#So sánh tốc độ của 2 phương pháp truy xuất word bằng index
t1 = time.time()
for i in range(1000):
  words_list.index('ngon')
print('Time using index:', time.time()-t1)

t2 = time.time()
for i in range(1000):
  word2idx['ngon']
print('Time using dictionary:', time.time()-t2)

### Tiền xử lý văn bản

Để tiết kiệm thời gian, tập dữ liệu được cung cấp đã được **tách từ** sẵn. Những từ ghép gồm nhiều tiếng sẽ có dấu '_' nối giữa các tiếng, ví dụ: 'sinh_viên', 'máy_chiếu_hình'.

> *Tách từ tiếng Việt* (Vietnamese Word Segmentation) 
là một bài toán không đơn giản, và cũng **nằm ngoài phạm vi của khóa học này**. Trong tiếng Anh, hầu hết các từ đều được tách bằng dấu cách. Tuy nhiên, trong tiếng Việt, dấu cách được dùng để tách biệt các **tiếng** chứ không phải các **từ**, nên đây là một vấn đề quan trọng cần lưu ý khi xử lý văn bản tiếng Việt.



Ngoài ra, bạn cũng được cung cấp sẵn các hàm nhằm loại bỏ các ký tự đặc biệt trong văn bản. Tham khảo hàm `clean_sentences` ở dưới.

In [ ]:
# Loại bỏ các dấu câu, dấu ngoặc, chấm than chấm hỏi, vân vân..., chỉ chừa lại các kí tự chữ và số
import re
# re = regular expressions
strip_special_chars = re.compile("[^\w0-9 ]+")

def clean_sentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

### TODO 1: Viết hàm biểu diễn văn bản thành ma trận biểu diễn từ

Tiến thêm một bước từ việc biểu diễn một từ bằng một vector, ở phần này ta sẽ biểu diễn **một câu có nhiều từ** dưới dạng **một ma trận có nhiều vector**.

Ví dụ: _"Món này ăn hoài không biết chán"_

Đầu tiên, với mỗi từ trong câu trên, ta tìm chỉ số (index) tương ứng của từ trong `words_list`, rồi lưu tất cả các chỉ số đó vào một vector đặt tên là `sentence_indices`.

Sau đó, chúng ta có thể sử dụng hàm tra cứu ma trận word embedding của thư viện Tensorflow `tf.nn.embedding_lookup` để tra các vector tại các chỉ số trong `sentence_indices`. 

Nếu ta đặt giới hạn độ dài tối đa của mỗi câu là 10 từ, thì **ma trận biểu diễn câu** sẽ có kích thước *10 x 300*, tương ứng với *10 từ*, mỗi từ là 1 vector *300 phần tử*. Dĩ nhiên, nếu trong tập dữ liệu tồn tại câu dài hơn 10 từ thì ta có thể nâng giới hạn này lên cho phù hợp.

![Embedding](https://drive.google.com/uc?export=view&id=1UvvIa22H3RpWCs_kkzYPprjAPY2-9Ov5)

**Lưu ý**: Trường hợp gặp từ nào không có trong `words_list`, là bộ "từ điển" chứa những từ mà ta "biết" (tức là có biểu diễn vector), thì ta sẽ gán cho từ đó chỉ số của từ `UNK` (unknown).


In [ ]:
word2idx['UNK']

In [ ]:
def get_sentence_indices(sentence, max_seq_length, _words_list):
    """
    Hàm này dùng để lấy index cho từng từ
    trong câu (không có dấu câu, có thể in hoa)
    Parameters
    ----------
    sentence là câu cần xử lý
    max_seq_length là giới hạn số từ tối đa trong câu
    _words_list là bản sao local của words_list, được truyền vào hàm
    """
    indices = np.zeros((max_seq_length), dtype='int32')
    
    # Tách câu thành từng từ
    words = [word.lower() for word in sentence.split()]
    
    # Lấy chỉ số của UNK
    unk_idx = word2idx['UNK']
    
    ### TODO 1 ###
    # Viết code để điền vào indices các giá trị index tương ứng trong words
    # LƯU Ý: len(indices) có thể ngắn hơn len(words)
    ### START CODE HERE ###
    
    ### END CODE HERE ###
    return indices

Ở ví dụ dưới đây, ta kiểm tra lại hàm vừa cài đặt ở TODO 1. Nếu bạn thực hiện đúng thì vector `sentence_indices` sẽ có giá trị là:

 `[11975,  8136, 19898, 18109, 10627, 12548, 19898,     0,     0,
           0]`

In [ ]:
# Ví dụ:
sentence = "Quán này bé_tí, nhưng siêu cute hạt_me."

# Tiền xử lý câu
sentence = clean_sentences(sentence)
print(sentence)

In [ ]:
sentence_indices = get_sentence_indices(sentence, max_seq_length=10, _words_list=words_list)
print(sentence_indices)

In [ ]:
# Ma trận biểu diễn:
print('Vector representation of sentence:')
print(tf.nn.embedding_lookup(word_vectors,sentence_indices))

### Khảo sát tập dữ liệu huấn luyện

Với khối lượng dữ liệu lớn (27,000 mẫu), nếu chúng ta chọn đặt **giới hạn số từ tối đa cho một câu** (`MAX_SEQ_LENGTH`) quá cao thì sẽ hao phí bộ nhớ khi biểu diễn những câu review quá ngắn. Ngược lại, nếu đặt giới hạn quá thấp thì ở những câu dài, ta sẽ bị mất các từ cuối, có khả năng ảnh hưởng đến việc phân tích cảm xúc.

Vì vậy, ta cần thống kê độ dài của các mẫu dữ liệu huấn luyện để lựa chọn giá trị hợp lý cho giới hạn này.


In [ ]:
num_words = [len(clean_sentences(x).split()) for x in list(train_df['text'])]
print('The total number of samples is', len(train_df))
print('The total number of words in the files is', sum(num_words))
print('The average number of words in the files is', sum(num_words)/len(num_words))

Chúng ta có thể sử dụng thư viện Matplotlib để minh họa phân bố về chiều dài của các câu review trong tập dữ liệu:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(num_words, 100)
plt.xlabel('Số từ trong câu')
plt.ylabel('Tần số')
plt.axis([0, 600, 0, 5000])
plt.show()

Dựa trên biểu đồ histogram ở trên chúng ta có thể thấy chọn đặt giới hạn `MAX_SEQ_LENGTH` 200 là tương đối hợp lý. 

In [ ]:
MAX_SEQ_LENGTH = 200

### Chuyển dữ liệu văn bản thành ma trận

Trong phần này, chúng ta sẽ tiến hành biến đổi text của review thành vector index của các từ trong `words_list`. Tập hợp các vector của `train_df` sẽ tạo thành ma trận `train_ids`

In [ ]:
def text2ids(df, max_length, _word_list):
    """
    Biến đổi các text trong dataframe thành ma trận index
    
    Parameters
    ----------
    df: DataFrame
        dataframe chứa các text cần biến đổi
    max_length: int
        độ dài tối đa của một text
    _word_list: numpy.array
        array chứa các từ trong word vectors
    
    Returns
    -------
    numpy.array
        len(df) x max_length contains indices of text
    """
    ids = np.zeros((len(df), max_length), dtype='int32')
    for idx, text in enumerate(tqdm(df['text'])):
        ids[idx,:] = get_sentence_indices(clean_sentences(text), max_length, _word_list)
    return ids

In [ ]:
# LƯU Ý: Bước thực hiện này tương đối mất thời gian (khoảng 5 phút)
# Nếu đã có file train_ids.npy rồi thì ra có thể load lên
# và sử dụng luôn ở bước tiếp theo

print("Converting train_df to train_ids...")
train_ids = text2ids(train_df, MAX_SEQ_LENGTH, words_list)
np.save('train_ids.npy', train_ids)


In [ ]:
# Trường hợp đã tính toán và lưu ma trận rồi thì ta có thể load lên
train_ids = np.load('train_ids.npy')

In thử word indices của review đầu tiên:

In [ ]:
print('Word indices of the first review: ')
print(train_ids[0])


### Chia dữ liệu thành train, validation và test

Chia dữ liệu `train_ids` và cột `class` của `train_df` thành 3 phần **train : validation : test = 0.8 : 0.1 : 0.1**

In [ ]:
train_x, test_validation_x, train_y, test_validation_y  = train_test_split(train_ids, train_df['class'], test_size=0.2, random_state=2019)
validation_x, test_x, validation_y, test_y = train_test_split(test_validation_x, test_validation_y, test_size=0.5, random_state=2018)

### TODO 2: Khởi tạo các `train_dataset`, `validation_dataset`, `test_dataset`

In [ ]:
BATCH_SIZE = 256 # Có thể chạy trên Tesla K80 12GB VRAM

In [ ]:
### TODO 2
# Sử dụng các biến đã tạo ở trên, hình thành 3 tập dataset
# train_dataset, validation_dataset, test_dataset
# trong đó mỗi tập dataset đều được chia thành các batch
#
# GỢI Ý:
# Cả 3 tập đều có quy trình tạo như nhau, chỉ khác về tham biến.
#
# Một dataset được tạo như sau:
# dataset_1 = tf.data.Dataset.from_tensor_slices((features,targets))
#
# Các dataset sau đó cần được chia thành batch.
# dataset_1 = dataset_1.batch(batch_size)
#
# Tham khảo: https://www.tensorflow.org/api_docs/python/tf/data/Dataset

### START CODE HERE

### END CODE HERE

In [ ]:
train_x.shape

Chạy dòng lệnh sau để kiểm tra TODO 2:

In [ ]:
for idx, (x,y) in enumerate(train_dataset):
    if idx == 0:
        print('FIRST BATCH:')
        print('X =',x)
        print('y =',y)
print("Total: ", idx)

Nếu cài đặt đúng, kết quả sẽ có dạng như sau:
```
FIRST BATCH:
X = tf.Tensor(
[[12844  5596  4884 ...     0     0     0]
 [10774 16521 13952 ...     0     0     0]
 [ 7446 10698 10774 ...     0     0     0]
 ...
 [ 4884 16995  4601 ...     0     0     0]
 [ 3913 15085 14017 ...     0     0     0]
 [ 4788 14598   310 ...     0     0     0]], shape=(256, 200), dtype=int32)
y = tf.Tensor(
[0 0 1 0 0 1 1 0 0 1 1 1 0 0 1 1 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0 1 0 0 1 1 0
 0 0 1 0 1 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 1 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0
 0 1 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 1 1 0 0 0 1 0 1 0 1 1 1 0 0 1 0 0 1 1 1
 0 1 1 0 1 0 0 0 1 1 0 1 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 1 1 1 1 1 1
 0 0 0 0 0 0 1 1 1 1 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1 0 0 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 1 0
 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 1 1 1 0 1 1], shape=(256,), dtype=int32)
Total:  84
```

In [ ]:
21600/BATCH_SIZE

> *Câu hỏi phụ: số 84 trong kết quả trên nghĩa là gì? Liên hệ trở lại với BATCH_SIZE.*



### Tổng kết quá trình chuẩn bị dữ liệu huấn luyện

Quá trình kết hợp ma trận chỉ số (indices) và ma trận biểu diễn từ (word embeddings) sang data tensor dùng để huấn luyện được mô tả như hình dưới đây:
![caption](https://drive.google.com/uc?export=view&id=1SST9hIt4jboWYr4A41-wI2jMbKI25A9e)

Từ ma trận indices và ma trận embedding, sử dụng `tf.nn.embedding_lookup` ta thu được tensor có kích thước `(BATCH_SIZE, MAX_SEQUENCE_LENGTH, WORD_VEC_DIM)`.


In [ ]:
train_x[0]

In [ ]:
tf.nn.embedding_lookup(word_vectors, train_x[0])

In [ ]:
for idx, (x,y) in enumerate(train_dataset):
    if idx == 0:
        print('EMBEDDING OF FIRST BATCH:')
        print(tf.nn.embedding_lookup(word_vectors, x))

Chú ý vào phần shape của tensor output ở trên, ta có thể thấy kích thước của từng batch dữ liệu, tương ứng với hình minh họa trên.

Vậy là ta đã hoàn thành quá trình chuẩn bị dữ liệu!

Tiếp theo, ta sẽ xây dựng mô hình RNN để huấn luyện trên dữ liệu này.

## Xây dựng mô hình RNN

Trong bài tập này, ta sẽ sử dụng mạng Long Short-Term Memory (LSTM), một dạng kiến trúc RNN để gỉải quyết bài toán phân tích cảm xúc.

**Kiến trúc mạng LSTM** ta sẽ sử dụng trong bài tập này được mô tả như hình dưới. Để tăng độ phức tạp của mô hình, ta chồng nhiều tầng LSTM lên nhau (**Stacked LSTM layers**). Giữa tầng LSTM dưới và tầng LSTM trên, ta đặt một lớp dropout.

Việc chồng thêm các tầng LSTM sẽ giúp cho mô hình có khả năng học được các biểu diễn phức tạp hơn (tương tự như khi dùng nhiều tầng CNN), nhưng đồng thời cũng làm tăng số lượng tham số, đồng nghĩa với việc làm tăng thời gian huấn luyện.

![caption](https://drive.google.com/uc?export=view&id=10m5sPPmba__bHx6V9Wp3fp1Wot5uOhBG)

### Chọn  LSTM layer

Để hỗ trợ tốt nhất cho việc tính toán trên GPU, keras hỗ trợ hai loại LSTM layer: 1 loại dành được tối ưu cho các tính toán trên GPU, và một loại dành cho CPU.

In [ ]:
# NEW KERAS LSTM LAYER (suitable for both CPU and GPU): https://keras.io/api/layers/recurrent_layers/lstm/
lstm_layer = tf.keras.layers.LSTM

### TODO 3: Định nghĩa và tạo mô hình
Ở bước này, chúng ta cần định nghĩa mô hình `SentimentAnalysisModel`, kế thừa từ `tf.keras.Model`. Chúng ta sẽ áp dụng mô hình stacked LSTM ở trên để xây dựng `SentimentAnalysisModel`.

Các bạn theo dõi hướng dẫn cụ thể trong code ở dưới.

Hoàn thành hàm `__init__` để tạo định nghĩa của `class SentimentAnalysisModel`.

Hoàn thành hàm `call` để tạo lệnh feedforward khi gọi một `object` của `class SentimentAnalysisModel`.

In [ ]:
class SentimentAnalysisModel(tf.keras.Model):
    """
    Mô hình phân tích cảm xúc của câu
    
    Properties
    ----------
    word2vec: numpy.array
        word vectors 
    lstm_layers: list
        list of lstm layers, lstm cuối cùng sẽ chỉ trả về output của lstm cuối cùng
    dropout_layers: list
        list of dropout layers
    dense_layer: Keras Dense Layer
        lớp dense layer cuối cùng nhận input từ lstm, 
        đưa ra output bằng số lượng class thông qua hàm softmax
    """
    def __init__(self, word2vec, lstm_units, n_layers, num_classes, dropout_rate=0.25):
        """
        Khởi tạo mô hình
        
        Paramters
        ---------
        word2vec: numpy.array
            word vectors 
        lstm_units: int
            số đơn vị lstm
        n_layers: int
            số layer lstm xếp chồng lên nhau
        num_classes: int
            số class đầu ra
        dropout_rate: float
            tỉ lệ dropout giữa các lớp
        """
        super().__init__(name='sentiment_analysis')
        
        # Khởi tạo các đặc tính của model
        self.word2vec = word2vec
        
        self.lstm_layers = []  # List chứa các tầng LSTM
        self.dropout_layers = []  # List chứa các tầng dropout

        ### TODO 3.1
        # Viết vòng lặp chạy qua N tầng trong stack
        # mỗi tầng sẽ có 1 lstm_layer và 1 dropout_layer
        # 
        # Khởi tạo lstm_layer như sau: 
        # new_lstm = tf.keras.layers.LSTM(units=..., return_sequence=... )
        # Tham khảo: https://keras.io/api/layers/recurrent_layers/lstm/
        # LƯU Ý:
        # return_sequence của lstm_layer nhận giá trị True ở mọi tầng  
        # trong stack, ngoại trừ tầng cuối cùngg
        #
        # Khởi tạo dropout_layer như sau:
        # new_dropout = tf.keras.layers.Dropout(rate=...)
        # Tham khảo: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout
        # 
        # Sau khi khởi tạo lstm_layer và dropout_layer
        # hãy thêm chúng vào 2 list tương ứng
        # self.lstm_layers và self.dropout_layers
        #
        # Cuối cùng, khởi tạo tầng fully-connected/dense
        # tf.keras.layers.Dense(num_classes=..., activation=' ')
        # Tham khảo: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
        #
        ### START CODE HERE
        
        ### END CODE HERE
        
    def call(self, inputs):
        ### TODO 3.2
        # Thực hiện các bước biến đổi khi truyền thuận input qua mạng
        # Điền code thay vào các đoạn pass
        ### START CODE HERE
        pass
        # Input hiện là indices, cần chuyển sang dạng vector
        # sử dụng:
        # tf.nn.embedding_lookup(embeddings, indices)
        pass
        # Truyền thuận inputs lần lượt qua các tầng
        # ở mỗi tầng, truyền input qua các layer: lstm > dropout
        # vd: x = first_lstm(x)
        #     x = first_dropout(x)
        #     x = second_lstm(x)
        #     v.v.
        pass
        # Gán giá trị tầng cuối cùng vào out và trả về
        pass
        
        ### END CODE HERE
        return output

Tiếp theo, ta lựa chọn các hyperparameters (siêu tham số) phù hợp, rồi tạo mô hình dựa trên định nghĩa.

In [ ]:
# Các hyperparameters
LSTM_UNITS = 128
N_LAYERS = 2
NUM_CLASSES = 2

In [ ]:
model = SentimentAnalysisModel(word_vectors, LSTM_UNITS, N_LAYERS, NUM_CLASSES)

##### SANITY CHECK FOR TODO3

In [ ]:
# Expected output is a Tensor of shape (5,2)
# corresponding to the probability of classes 0 and 1, ie. negative and positive, for 5 samples
model(train_x[0:5])

> *Câu hỏi phụ: Hãy thử ước tính xem mô hình với các siêu tham số trên sẽ có bao nhiêu tham số cần huấn luyện, với kích thước input là 200? Kiểm tra bằng `model.summary()`.*


### Huấn luyện mô hình

Ở bài tập 3 (CNN), ta đã sử dụng hàm `fit` của keras model trong quá trình huấn luyện.

Ở bài tập này, quá trình huấn luyện được thực hiện ở cấp độ thấp hơn (lower level), và ta sẽ đi qua từng bước: **truyền thuận**, **truyền ngược**, và **tối ưu hoá**

Đầu tiên, chọn các tham số huấn luyện:

In [ ]:
LEARNING_RATE = 0.001
EPOCHS = 100

Ta sẽ sử dụng thuật toán tối ưu hoá Adam với learning rate là `LEARNING_RATE`:

> Tham khảo thêm về Adam ở các link dưới:
>  * tiếng Anh: https://ml-cheatsheet.readthedocs.io/en/latest/optimizers.html#adam
>  * tiếng Anh: http://ruder.io/optimizing-gradient-descent/index.html#adam
>  * tiếng Việt: https://viblo.asia/p/thuat-toan-toi-uu-adam-aWj53k8Q56m



In [ ]:
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

Các biến khác hỗ trợ cho việc huấn luyện:

In [ ]:
# Thư mục chứa checkpoint
checkpoint_dir = './model'
# Mẫu tên checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{score}")

#### TODO 4: Viết hàm huấn luyện từng epoch

Trước tiên, ta sẽ import vài thành phần từ thư viện `fastprogress` để tiện cho việc biểu diễn 
quy trình huấn luyện:

In [ ]:
from fastprogress import master_bar, progress_bar

Ta cần xây dựng hàm `epoch_training`, thực hiện các bước **truyền thuận**, **truyền ngược** và **tối ưu hóa** tại mỗi epoch.

Hàm `epoch_training`:
* nhận các tham biến `model`, `dataset`, `mb`, và `num_step`
* trả về giá trị loss trung bình của model

In [ ]:
def epoch_training(model, dataset, mb, num_step):
    """
    Huấn luyện mô hình trong 1 epoch
    
    Parameters
    ----------
    model: tf.keras.Model
        model cần huấn luyện
    dataset: tf.Dataset
        dataset dùng để huấn luyện
    mb: fastprogress.master_bar
        thanh biểu diễn tiến độ
    num_step: int
        số bước lặp trong mỗi epoch
    Returns
    -------
    float
        Giá trị loss trung bình của epoch
    """
    train_losses = [] # Chứa giá trị loss của các batch
    dataset_iter = iter(dataset)
    for batch in progress_bar(range(num_step), parent=mb):
        inp, target = next(dataset_iter)
        ### TODO 4
        # tham khảo cách dùng tf.GradientTape ở def train_step()
        # ở link https://www.tensorflow.org/guide/migrate#customize_the_training_step
        with tf.GradientTape() as tape:
            
            ### TODO 4.1
            # Truyền thuận - Feedforward
            # Để tạo giá trị predictions
            ### START CODE HERE
            
            ### END CODE HERE
            
            ### TODO 4.2
            # Tính loss giữa target và predictions
            # sử dụng hàm loss sparse_softmax_cross_entropy
            ### START CODE HERE
            
            ### END CODE HERE
          
            # Thêm loss của batch vào train_losses
            train_losses.append(loss)
            
        
        
        ### TODO 4.3
        # Tính gradient của loss theo các tham số của mô hình
        ### START CODE HERE
        
        ### END CODE HERE
        
        ### TODO 4.4
        # Dùng gradient để tối ưu các tham số
        ### START CODE HERE
        
        ### END CODE HERE
        
        mb.child.comment = 'Train loss {:.4f}'.format(loss)
        
    # Trả về loss trung bình
    return sum(train_losses)/ len(train_losses)

#### SANITY CHECK FOR TODO 4

In [ ]:
# Test epoch_training() over one trial epoch
mb = master_bar(range(1))
for epoch in mb:
  print('Training loss after 1 trial epoch:', epoch_training(model, train_dataset, mb, math.ceil(len(train_x)*1.0/BATCH_SIZE)))

#### TODO 5: Viết hàm đánh giá từng epoch

Ta cần xây dựng hàm `epoch_evaluation`, thực hiện duy nhất bước **truyền thuận** để tính loss.

Hàm `epoch_evaluation`:
* nhận vào `model`, `dataset`, `mb`, và `num_step`
* trả về giá trị `loss`, `f1_score` của model trên tập `dataset` 

In [ ]:
def epoch_evaluation(model, dataset, mb, num_step):
    """
    Đánh giá mô hình qua 1 epoch
    
    Parameters
    ----------
    model: tf.keras.Model
        model cần đánh giá
    dataset: tf.Dataset
        dataset cần đánh giá
    mb: fastprogress.master_bar
        progess bar
    num_step: int
        số bước lặp trong mỗi epoch
    Returns
    -------
    float
        Giá trị loss trung bình của epoch
    float
        Giá trị F1-score trên tập validation
    """
    val_losses = []   # Chứa giá trị loss của các batch
    val_gt = []       # Chứa giá trị nhãn thật của tập validation , gt = groundtruth
    val_pred = []     # Chứa giá trị nhãn dự đoán do model trên tập validation
    dataset_iter = iter(dataset)
    for batch in progress_bar(range(num_step), parent=mb):
        inp, target = next(dataset_iter)
        
        val_gt.extend(target.numpy().astype(np.int32).tolist())
        ### TODO 5.1
        # Truyền thuận
        # để lấy predictions
        ### START CODE HERE
        
        ### END CODE HERE
        
        ### TODO 5.2
        # Tính loss giữa target và predictions
        # sử dụng hàm loss sparse_softmax_cross_entropy
        ### START CODE HERE
        
        ### END CODE HERE
        
        
        # Thêm loss vào val_losses
        val_losses.append(loss)
        val_pred.extend(tf.argmax(predictions,1).numpy().astype(np.int32).tolist())
        
        mb.child.comment = 'Validation loss {:.4f}'.format(loss)
    # Trả về loss trung bình và F1-score
    return sum(val_losses) / len(val_losses), f1_score(val_gt, val_pred)

#### SANITY CHECK FOR TODO 5

In [ ]:
# Test epoch_evaluation() over one trial epoch
mb = master_bar(range(1))
for epoch in mb:
  val_loss, val_score = epoch_evaluation(model, validation_dataset, mb, math.ceil(len(validation_x)*1.0/BATCH_SIZE))
print('Validation loss:', val_loss)
print('Validation F1 score:', val_score)

#### TODO 6: Thiết lập quy trình huấn luyện


Khi đã chuẩn bị xong hàm huấn luyện và đánh giá cho mỗi epoch, ta có thể thiết lập quy trình huấn luyện chính.

In [ ]:
best_score = 0
mb = master_bar(range(EPOCHS))
mb.names = ['Training loss', 'Validation loss', 'F1']
training_losses = []
validation_losses = []
f1_scores = []
x = []

train_step = math.ceil(len(train_x)*1.0/BATCH_SIZE)
val_step = math.ceil(len(validation_x)*1.0/BATCH_SIZE)
for epoch in mb:
    # initializing the hidden state at the start of every epoch
    # initally hidden is None
    hidden = model.reset_states()
    x.append(epoch)
    
    ### TODO 6.1
    # Huấn luyện bằng hàm epoch_training()
    # Sử dụng tập train_dataset
    ### START CODE HERE
    
    ### END CODE HERE
    
    # Báo cáo kết quả train/huấn luyện
    mb.write('Finish train epoch {} with loss {:.4f}'.format(epoch, training_loss))
    training_losses.append(training_loss)
    
    ### TODO 6.2
    # Đánh giá kết quả trên tập validation_dataset
    # bằng hàm epoch_evaluation()
    ### START CODE HERE
    
    ### END CODE HERE
    
    # Báo cáo kết quả validate
    mb.write('Finish validate epoch {} with loss {:.4f}, F1-score {:.4f}'.format(epoch,valid_loss, valid_score))
    validation_losses.append(valid_loss)
    f1_scores.append(valid_score)
    
    # Cập nhật đồ thị
    mb.update_graph([[x, training_losses], [x, validation_losses], [x, f1_scores]], [0,EPOCHS], [0,1.5])
    
    # Update score và lưu model có score tốt nhất
    if best_score < valid_score:
        mb.write(">>> Improved F1-score from {:.4f} to {:.4f}".format(best_score, valid_score))
        # Update best_score
        best_score = valid_score
        # Save model
        model.save_weights(checkpoint_prefix.format(score=valid_score))
        
    

### Đánh giá mô hình trên tập test có nhãn

Sau khi hoàn thành quá trình huấn luyện, ta sẽ đánh giá mô hình trên tập test đã chia ra từ dữ liệu có nhãn. Ta sử dụng thang điểm F1 để đánh giá độ chính xác của mô hình.

> Tham khảo về định nghĩa F1-score ở đây: https://machinelearningcoban.com/2017/08/31/evaluation/#-f-score
![alt text](https://machinelearningcoban.com/assets/33_evaluation/PR.png)

Nếu được huấn luyện đúng cách thì kết quả F1 ở phần này sẽ rơi vào khoảng 70-80%.

In [ ]:
mb = master_bar(range(1))
for i in mb:
  test_loss, test_score = epoch_evaluation(model, test_dataset, mb, math.ceil(len(test_x)*1.0/BATCH_SIZE))
print("F1-score on test set:", test_score)

## Predict trên 3000 mẫu bình luận không nhãn

Đã xong quá trình huấn luyện và kiểm định mô hình trên dữ liệu có nhãn.

Nếu thực hiện tốt các bài tập ở trên, thì đến đây bạn đã có một mô hình có thể phân tích cảm xúc tiêu cực/tích cực của một đoạn văn bản tiếng Việt bất kì!

(Tuy nhiên, nếu văn bản đó không phải là bình luận về địa điểm ăn uống thì mô hình sẽ khó lòng dự đoán chính xác, bởi dữ liệu mà ta dùng để huấn luyện chỉ nằm trong miền các bình luận đánh giá quán ăn. Có thể nói là mô hình của chúng ta chỉ có tri thức chuyên biệt về địa điểm ăn uống.)

Việc cuối cùng là áp dụng mô hình để suy đoán cảm xúc của các mẫu bình luận không nhãn.

In [ ]:
#Các bình luận không nhãn
test_df.head(10)

### TODO 7: Xây dựng hàm `predict`

Ta cần viết một hàm nhận vào một câu/đoạn bình luận tiếng Việt, và trả về một trong 2 giá trị cảm xúc positive hoặc negative.

Cần lưu ý: các dữ liệu không nhãn chưa được tách thành từ nên các bạn cần sử dụng các thư viện hỗ trợ tokenize tiếng Việt (ví dụ underthesea) để tiền xử lý trong hàm `predict`

In [ ]:
try:
    from underthesea import word_tokenize
except:
    !pip install underthesea
    from underthesea import word_tokenize

In [ ]:
def predict(sentence, model, _word_list=words_list, _max_seq_length=MAX_SEQ_LENGTH):
    """
    Dự đoán cảm xúc của một câu
    
    Parameters
    ----------
    sentence: str
        câu cần dự đoán
    model: model keras
        model keras đã được train/ load trọng số vừa train
    _word_list: numpy.array
        danh sách các từ đã biết
    _max_seq_length: int
        giới hạn số từ tối đa trong mỗi câu
        
    Returns
    -------
    int
        0 nếu là negative, 1 nếu là positive
    """
    ### TODO 7.1
    # Tokenize/Tách từ trong câu
    # Sử dụng hàm word_tokenize vừa import ở trên
    ### START CODE HERE
    
    ### END CODE HERE
    
    ### TODO 7.2
    # Đưa câu đã tokenize về dạng input_data thích hợp để truyền vào model
    ### START CODE HER
    
    ### END CODE HERE
    
    
    ### TODO 7.3
    # Truyền input_data qua model để nhận về xác suất các nhãn
    # Chọn nhãn có xác suất cao nhất và return
    ### START CODE HERE
    
    ### END CODE HERE
    return predictions

Load trọng số model đã train:

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

Kiểm tra hàm vừa cài đặt, nếu cài đặt hàm và huấn luyện đúng thì kết quả lần lượt **_nên_** là 0 và 1.

In [ ]:
print("Test 1:",predict("Quán này rất dở", model))
print("Test 2:",predict("Quán này rất ngon", model))

### Ghi kết quả

Đọc file `sample_submission.csv`

In [ ]:
submission = pd.read_csv(data_dir+"/sample_submission.csv", index_col=0)
submission.head()

Duyệt từng dòng trong DataFrame `test_df` và dự đoán sau đó cập nhật kết quả vào `submission`

In [ ]:
for _, row in tqdm(test_df.iterrows()):
    submission.loc[row.id] = predict(row.text, model)

In [ ]:
submission.head()

### Nộp bài

Xuất ra file `submission.csv`.

In [ ]:
submission.to_csv('submission.csv')

* Nộp Notebook: https://forms.gle/KsCYTRLebTBR3omW9
* Nộp file submission lên [Kaggle competition](https://www.kaggle.com/t/4af2ae58e2294153931a839a75eece16)

Hạn nộp bài: Trước 23:59, 05-08-2020